In [1]:
import sys
import os
import numpy as np
import pandas as pd
from src import multivariate_os, nominal_os
from collections import Counter
from sklearn.model_selection import train_test_split

In [2]:
# Multivariate over-sampling
def mndo(pos, num_minority):
    pos, zero_std = multivariate_os.find_zerostd(pos, num_minority)
    pos, no_corr = multivariate_os.no_corr(pos, num_minority)
    pos = multivariate_os.mnd_os(pos, num_minority)
    #numerical_os.nc_os(pos, zero_std, no_corr, nominal)
    #mndo_df = multivariate_os.append_data(pos, zero_std, no_corr, name)
    return pos

In [30]:
# Load dataset
try:
    data = pd.read_csv('Predataset/abalone_19/continuous.csv')
    nominal = pd.read_csv('Predataset/abalone_19/nominal.csv')
    #data = pd.read_csv('Predataset/{}/continuous.csv'.format(sys.argv[1]))
    #nominal = pd.read_csv('Predataset/{}/nominal.csv'.format(sys.argv[1]))
    #file_name = sys.argv[1]
except IndexError:
    sys.exit('error: Must specify dataset file')
except FileNotFoundError:
    sys.exit('error: No such file or directory')

In [32]:
# split the data
X = data.drop('Label', axis=1)
y = data.Label
X = np.array(X)
y = np.array(y)

# split positive class
pos = data[data.Label == 1]
pos = pos.drop('Label', axis=1)
nominal = nominal[nominal.Label == 1]
nominal = nominal.drop('Label', axis=1)


# calc number of  samples to synthesize
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
        shuffle=True, random_state=42)

cnt = Counter(y_train)
num_minority = int((cnt[-1] - cnt[1]))

In [5]:
# over-sampling
pos_gen = mndo(pos, num_minority)

Searching no correlation:  11%|█         | 11/100 [00:00<00:00, 107.27it/s]

Not found zero std.


Multi normal dist over-sampling:   0%|          | 0/100 [00:00<?, ?it/s]

Not found no correlation.


In [6]:
print(num_minority)
print(pos.shape)
print(pos_gen.shape)

2875
(32, 7)
(2875, 7)


In [7]:
distance = []
for i in range(len(pos_gen)):
    diff = np.array(pos_gen.iloc[i]) - np.array(pos)
    tmp = [np.linalg.norm(diff[j]) for j in range(len(diff))]
    distance.append(tmp)
del tmp

In [96]:
df_dist = pd.DataFrame(distance).T
df_dist.index = pos.index

In [97]:
df_dist.head()

,0,1,2,3,4,5,6,7,8,9,...,2865,2866,2867,2868,2869,2870,2871,2872,2873,2874
9,0.563803,1.051397,0.355013,0.301597,0.365890,0.497420,0.393342,0.147347,0.359769,1.017413,...,0.387243,0.241516,0.469838,0.454921,0.846660,0.654317,0.317906,0.604407,0.398120,0.931576
33,0.719568,0.461714,1.007662,0.841525,1.218238,0.958734,1.424635,1.109105,1.271991,0.460430,...,1.283110,1.003208,1.330205,0.938712,0.344621,0.803733,0.839844,0.635062,0.822589,0.356302
128,0.537898,0.422606,0.831792,0.725331,1.124441,0.911401,1.290547,0.959659,1.155108,0.371712,...,1.193110,0.851202,1.239594,0.752299,0.309997,0.622011,0.740913,0.459293,0.718891,0.219659
256,0.413226,0.881310,0.322140,0.169787,0.482204,0.464272,0.584626,0.248815,0.490188,0.840776,...,0.536500,0.253089,0.590938,0.372087,0.651669,0.525760,0.233471,0.436302,0.287696,0.738357
293,0.452591,0.927254,0.296322,0.213635,0.445813,0.470972,0.510362,0.212698,0.456887,0.903132,...,0.459098,0.238794,0.544879,0.370973,0.715444,0.565450,0.211978,0.486179,0.330420,0.801087


In [154]:
key = []
for i in range(len(df_dist.columns)):
    sort = np.argsort(-df_dist[df_dist.columns[i]])
    key.append(sort[0:5])

key = np.array(key)
"""rank = []
for i in range(len(key)):
    rank.append(df_dist.index[key[i]])
    
rank = np.array(rank.T)
"""

'rank = []\nfor i in range(len(key)):\n    rank.append(df_dist.index[key[i]])\n    \nrank = np.array(rank.T)\n'

In [156]:
key[0]

array([15, 22, 17, 18, 24])

In [135]:
nominal.head()

,0,1,2
9,1.0,0.0,0.0
33,1.0,0.0,0.0
128,0.0,0.0,1.0
256,0.0,0.0,1.0
293,1.0,0.0,0.0


In [160]:
nominal.iloc[15]

0    0.0
1    1.0
2    0.0
Name: 664, dtype: float64

In [169]:
print(key.shape[0])
print(key.shape[1])

2875
5


In [213]:
tmp = []
for i in range(1):
    for j in range(5):
        tmp.append(nominal.iloc[key[0][j]])

In [214]:
aaa = pd.DataFrame(tmp)
aaa.head()

,0,1,2
664,0.0,1.0,0.0
2366,0.0,0.0,1.0
2101,0.0,0.0,1.0
2160,0.0,0.0,1.0
3244,1.0,0.0,0.0


In [219]:
nominal_gen = []

for i in range(len(aaa.columns)):
    counter = Counter(aaa[aaa.columns[i]])
    print(counter)
    
    # value, count
    b, c = counter.most_common(1)[0]
    nominal_gen.append(b)

Counter({0.0: 4, 1.0: 1})
Counter({0.0: 4, 1.0: 1})
Counter({1.0: 3, 0.0: 2})


In [220]:
nominal_gen

[0.0, 0.0, 1.0]